In [6]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# -----------------------------
# helpers
# -----------------------------
def add_intercept(X: np.ndarray) -> np.ndarray:
    return np.c_[np.ones((X.shape[0], 1)), X]

def mse(y: np.ndarray, yhat: np.ndarray) -> float:
    return float(mean_squared_error(y, yhat))

def ridge_gradient_descent(Xb: np.ndarray, y: np.ndarray, alpha: float, num_iters: int, lam: float) -> np.ndarray:
    """
    Minimize:
      J(theta) = sum_i (x_i^T theta - y_i)^2 + lam * sum_{j>=1} theta_j^2
    using batch gradient descent.

    Gradient:
      grad = 2 X^T (X theta - y) + 2 lam [0, theta_1, ..., theta_d]
    """
    N, d = Xb.shape
    theta = np.zeros(d, dtype=float)

    for _ in range(num_iters):
        preds = Xb @ theta                  # (N,)
        err = preds - y                     # (N,)

        grad = (2.0 / N) * (Xb.T @ err)     # (d,)

        # ridge penalty (do NOT penalize intercept)
        grad[1:] += (2.0 * lam / N) * theta[1:]

        theta -= alpha * grad

    return theta

def run_simulation(alpha=0.01, num_iters=20000, seed=0):
    np.random.seed(seed)

    # ---------------------------
    # Problem 6.3: simulate data
    # ---------------------------
    N = 1000
    X = np.random.uniform(-2, 2, size=(N, 1))
    e = np.random.normal(0, 2, size=N)
    y = 1 + 2 * X[:, 0] + e

    # IMPORTANT for stability: standardize X (this keeps GD from exploding)
    X_mean = X.mean(axis=0)
    X_std = X.std(axis=0)
    Xs = (X - X_mean) / X_std

    Xb = add_intercept(Xs)

    def predict(Xb, theta):
        return Xb @ theta

    # Linear regression = ridge with lambda=0
    theta_lin = ridge_gradient_descent(Xb, y, alpha=alpha, num_iters=num_iters, lam=0.0)
    yhat_lin = predict(Xb, theta_lin)

    print(f"=== Linear Regression (lambda=0) ===")
    print("theta:", theta_lin)
    print("slope (w.r.t standardized X):", theta_lin[1])
    print("MSE:", mse(y, yhat_lin))
    print("R2:", r2_score(y, yhat_lin))
    print()

    # Ridge for different lambdas
    lambdas = [1, 10, 100, 1000, 10000]
    for lam in lambdas:
        theta_r = ridge_gradient_descent(Xb, y, alpha=alpha, num_iters=num_iters, lam=lam)
        yhat_r = predict(Xb, theta_r)

        print(f"=== Ridge (lambda={lam}) ===")
        print("theta:", theta_r)
        print("slope (w.r.t standardized X):", theta_r[1])
        print("MSE:", mse(y, yhat_r))
        print("R2:", r2_score(y, yhat_r))
        print()

if __name__ == "__main__":
    # This setting is stable and will NOT overflow.
    run_simulation(alpha=0.01, num_iters=20000, seed=0)


=== Linear Regression (lambda=0) ===
theta: [1.02546064 2.26839267]
slope (w.r.t standardized X): 2.2683926678783237
MSE: 3.7307480978868743
R2: 0.57969811109718

=== Ridge (lambda=1) ===
theta: [1.02546064 2.26612654]
slope (w.r.t standardized X): 2.2661265413369867
MSE: 3.7307532332163764
R2: 0.5796975325567282

=== Ridge (lambda=10) ===
theta: [1.02546064 2.24593333]
slope (w.r.t standardized X): 2.2459333345329937
MSE: 3.731252519541192
R2: 0.5796412835203641

=== Ridge (lambda=100) ===
theta: [1.02546064 2.06217515]
slope (w.r.t standardized X): 2.0621751526166583
MSE: 3.7732737614875744
R2: 0.5749072176170374

=== Ridge (lambda=1000) ===
theta: [1.02546064 1.13419633]
slope (w.r.t standardized X): 1.1341963339391663
MSE: 5.0171494218079244
R2: 0.4347735833228846

=== Ridge (lambda=10000) ===
theta: [1.02546064 0.20621752]
slope (w.r.t standardized X): 0.20621751526166687
MSE: 7.983314457956452
R2: 0.1006087630829815

